Este programa está hecho para sacar los datos de Alphavantage

In [3]:
import requests
import time
from datetime import datetime, timedelta

def get_data(symbol, interval, month):
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={interval}&month={month}&outputsize=full&apikey=RFER9YLX4X9XH3RQ&datatype=csv"
    response = requests.get(url)
    print(url)
    return response.text
    
    #return "2024-09-11 19:55:00,222.0800,222.1445,222.0000,222.1445,4144"
    

def process_line(line, is_header=False):
    fields = line.strip().split(',')
    if is_header:
        return ','.join(fields)
    else:
        fields[-1] = str(int(fields[-1].strip('"')))
        return ','.join(fields)

symbol = "NVDA"
interval = "60min"
#start_date = datetime.now().replace(day=1) - timedelta(days=1)  # Último día del mes pasado
year = 2024 # Año de inicio
month = 10 # Mes de inicio
first_day_next_month = datetime(year, month, 1).replace(month=month % 12 + 1, day=1)
# Restar un día para obtener el último día del mes deseado
start_date = first_day_next_month - timedelta(days=1)
data_points = 0
filename = f"{symbol}_{interval}_data_{year}.csv" # Nombre del fichero
months = 12
cicles = 0

with open(filename, 'w', newline='') as csvfile:
    header_written = False
    last_timestamp = None

    while cicles < months:
        
        cicles += 1
        print('Ciclo ', cicles)    
        
        month = start_date.strftime("%Y-%m")
        csv_data = get_data(symbol, interval, month)
        csv_lines = csv_data.strip().split('\n')
        
        if not header_written:
            csvfile.write(process_line(csv_lines[0], is_header=True) + '\n')
            header_written = True
            csv_lines = csv_lines[1:]  # Skip header in data processing
        
        new_data_points = 0
        for line in csv_lines:
            current_timestamp = line.split(',')[0]
            if last_timestamp and current_timestamp >= last_timestamp:
                continue  # Skip duplicate data
            
            processed_line = process_line(line)
            csvfile.write(processed_line + '\n')
            new_data_points += 1
        
        if new_data_points == 0:
            print(f"No new data available for {month}. Moving to previous month.")
            start = start_date.replace(day=1) - timedelta(days=1)  # Move to previous month
            continue
        
        data_points += new_data_points
        last_timestamp = csv_lines[-1].split(',')[0]  # Update to the oldest timestamp in this batch
        
        print(f"Datos obtenidos para {month}. Total de puntos: {data_points}")

        start_date = start_date.replace(day=1) - timedelta(days=1)  # Move to previous month
        time.sleep(12)  # Esperar 12 segundos entre solicitudes

print(f"Proceso completado. Se han guardado {data_points} puntos de datos en {filename}")



Ciclo  1
https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=NVDA&interval=60min&month=2024-10&outputsize=full&apikey=RFER9YLX4X9XH3RQ&datatype=csv
Datos obtenidos para 2024-10. Total de puntos: 368
Ciclo  2
https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=NVDA&interval=60min&month=2024-09&outputsize=full&apikey=RFER9YLX4X9XH3RQ&datatype=csv
Datos obtenidos para 2024-09. Total de puntos: 688
Ciclo  3
https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=NVDA&interval=60min&month=2024-08&outputsize=full&apikey=RFER9YLX4X9XH3RQ&datatype=csv
Datos obtenidos para 2024-08. Total de puntos: 1040
Ciclo  4
https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=NVDA&interval=60min&month=2024-07&outputsize=full&apikey=RFER9YLX4X9XH3RQ&datatype=csv
Datos obtenidos para 2024-07. Total de puntos: 1389
Ciclo  5
https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=NVDA&interval=60min&month=2024-06&outputsize=f

In [18]:
!nvidia-smi

Tue Sep 24 18:17:13 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 552.44                 Driver Version: 552.44         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 850A      WDDM  |   00000000:08:00.0 Off |                  N/A |
| N/A    0C    P8             N/A /   32W |       0MiB /   2048MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [19]:
from tensorflow.python.client import device_lib

# Muestra información sobre los dispositivos disponibles
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13439965297162967527
xla_global_id: -1
]


In [17]:
import tensorflow as tf
print("GPUs disponibles:", tf.config.list_physical_devices('GPU'))

GPUs disponibles: []


In [7]:
#start_date = datetime.now().replace(day=1) - timedelta(days=1)  # Último día del mes pasado
year = 2024 # Año de inicio
month = 10 # Mes de inicio
first_day_next_month = datetime(year, month, 1).replace(month=month % 12 + 1, day=1)
# Restar un día para obtener el último día del mes deseado
start_date = first_day_next_month - timedelta(days=1)
print(start_date)

2024-10-31 00:00:00
